pip install tabula-py pandas requests beautifulsoup4

pip install pdfplumber
pip install pdfplumber

In [ ]:
#Importing the Libraries 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import pdfplumber
import io

/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [ ]:
#Step 1 - Helper Function (Define a function to clean up).
def clean_price(price_str):
    if isinstance(price_str, str):
        cleaned_str = price_str.replace(" ","")
        try:
            return float(cleaned_str)
        except ValueError:
            return pd.Na
    return price_str

In [ ]:
#Step 2 - Url 
base_url = 'https://www.ppac.gov.in/'
page_url = "https://ppac.gov.in/retail-selling-price-rsp-of-petrol-diesel-and-domestic-lpg/rsp-of-petrol-and-diesel-in-metro-cities-since-16-6-2017"

In [ ]:
#Step 3 - Scraping and Parsing
try:
    response = requests.get(page_url) #download the content.
    response.raise_for_status() #check for successful download no 404 errors.
    soup = BeautifulSoup(response.text, 'html.parser') #parse the content.
    pdf_link_tag = soup.find('a', href=lambda href: href and 'DailyPriceMSHSD_Metro' in href)
    
    if not pdf_link_tag:
        print("Error: Could not find the PDF link on the page.")
        exit()

    pdf_relative_url = pdf_link_tag['href']
    pdf_full_url = urljoin(f"https://{base_url}", pdf_relative_url)
    print(f"Found PDF at: {pdf_full_url}")

    pdf_response = requests.get(pdf_full_url)
    pdf_response.raise_for_status()
    print("Attempting to read tables from ALL pages...")
    
    all_data_rows = [] #A list to store all data rows from all pages.
    header = None  #A variable to store the column headers.

    with pdfplumber.open(io.BytesIO(pdf_response.content)) as pdf:
        try:
            first_page_tables = pdf.pages[0].extract_tables()
            header = first_page_tables[0][3]
            print(f"Successfully extracted header: {header}")
        except Exception as e:
            print(f"Fatal Error: Could not extract header from page 1. {e}") #If the header can't be found, the script can't continue.
            exit()
        for i, page in enumerate(pdf.pages): #Loop
            print(f"Processing page {i+1} of {len(pdf.pages)}...")
            tables = page.extract_tables()
            
            if not tables or len(tables) < 1:
                print(f"  - No tables found on page {i+1}.")
                continue
                
            combined_table_data = tables[0]
            data_on_this_page = combined_table_data[6:] 

            
            all_data_rows.extend(data_on_this_page) 
            print(f"  - Added {len(data_on_this_page)} data rows.")

    print(f"\nTotal data rows found across all pages: {len(all_data_rows)}")

    if not all_data_rows:
        print("Error: No data rows were found in the entire PDF.")
        exit()
except requests.exceptions.RequestException as e:
    print(f"Error fetching the URL: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Found PDF at: https://ppac.gov.in/uploads/page-images/1761110504_PP_9_a_DailyPriceMSHSD_Metro_22.10.2025.pdf
Attempting to read tables from ALL pages...
Successfully extracted header: ['', 'Date of\nRevision', 'Delhi', 'Mumbai', 'Chennai', 'Kolkata', '', 'Date of Revision', 'Delhi', 'Mumbai', 'Chennai', 'Kolkata', '']
Processing page 1 of 35...
  - Added 97 data rows.
Processing page 2 of 35...
  - Added 91 data rows.
Processing page 3 of 35...
  - Added 91 data rows.
Processing page 4 of 35...
  - Added 91 data rows.
Processing page 5 of 35...
  - Added 91 data rows.
Processing page 6 of 35...
  - Added 91 data rows.
Processing page 7 of 35...
  - Added 91 data rows.
Processing page 8 of 35...
  - Added 91 data rows.
Processing page 9 of 35...
  - Added 91 data rows.
Processing page 10 of 35...
  - Added 91 data rows.
Processing page 11 of 35...
  - Added 91 data rows.
Processing page 12 of 35...
  - Added 91 data rows.
Processing page 13 of 35...
  - Added 89 data rows.
Processing pa

In [ ]:
#Step 4 - Data Restructuring and Cleaning
#Splitting the dataframe 
full_df = pd.DataFrame(all_data_rows, columns=header)
petrol_df = full_df.iloc[:, 1:6] 
diesel_df = full_df.iloc[:, 7:12]

clean_columns = ['Date', 'Delhi', 'Mumbai', 'Chennai', 'Kolkata']
petrol_df.columns = clean_columns
diesel_df.columns = clean_columns

In [ ]:
#Reshaping the dataframe 
petrol_long = pd.melt(petrol_df, 
                          id_vars=['Date'], 
                          value_vars=['Delhi', 'Mumbai', 'Chennai', 'Kolkata'],
                          var_name='City', 
                          value_name='Petrol_Price')
    
diesel_long = pd.melt(diesel_df, 
                          id_vars=['Date'], 
                          value_vars=['Delhi', 'Mumbai', 'Chennai', 'Kolkata'],
                          var_name='City', 
                          value_name='Diesel_Price')
    
final_df = pd.merge(petrol_long, diesel_long, on=['Date', 'City']) #Merging the Dataframes 

In [ ]:
#Step 5 - Data Type Conversion.
final_df['Date'] = final_df['Date'].str.split('\n').str[0].str.strip()
final_df['Date'] = pd.to_datetime(final_df['Date'], format='%d-%b-%y')
final_df['Petrol_Price'] = final_df['Petrol_Price'].apply(clean_price)
final_df['Diesel_Price'] = final_df['Diesel_Price'].apply(clean_price)

In [8]:
final_df.dropna(subset=['Petrol_Price', 'Diesel_Price'], how='all', inplace=True)
final_df.sort_values(by=['Date', 'City'], ascending=[False, True], inplace=True)
final_df.reset_index(drop=True, inplace=True)

In [9]:
print("\nDataFrame Info:")
final_df.info()
    
print("\nDataFrame Head (First 5 Rows):")
print(final_df.head())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12068 entries, 0 to 12067
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          12068 non-null  datetime64[ns]
 1   City          12068 non-null  object        
 2   Petrol_Price  12068 non-null  float64       
 3   Diesel_Price  12068 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 377.2+ KB

DataFrame Head (First 5 Rows):
        Date     City  Petrol_Price  Diesel_Price
0 2025-10-22  Chennai        100.80         92.39
1 2025-10-22    Delhi         94.77         87.67
2 2025-10-22  Kolkata        105.41         92.02
3 2025-10-22   Mumbai        103.50         90.03
4 2025-10-21  Chennai        100.80         92.39


In [10]:
final_df.tail()

,Date,City,Petrol_Price,Diesel_Price
12063,2017-06-17,Mumbai,76.46,59.73
12064,2017-06-16,Chennai,68.02,57.41
12065,2017-06-16,Delhi,65.48,54.49
12066,2017-06-16,Kolkata,68.03,56.65
12067,2017-06-16,Mumbai,76.70,59.90


In [11]:
final_df.to_csv('fuel_prices.csv', index=False)
print("\nSuccessfully saved all data to 'fuel_prics.csv'")


Successfully saved all data to 'fuel_prics.csv'
